In [1]:
# Import packages

import numpy as np
import pandas as pd
from pandas_datareader import data
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Question II

In [3]:
### Import data

start = datetime.strptime('2004-01-02','%Y-%m-%d')
end = datetime.strptime('2009-6-30','%Y-%m-%d')

df = data.DataReader('^HSI',start = start, end = end, data_source = 'yahoo')
df_close = df[:1500]['Close']

In [4]:
### Map all data into [0,1]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))
data = scaler.fit_transform(np.array(df_close).reshape(-1,1))

In [5]:
### Split out training and test datasets

training_size = int(len(data)*0.65)
test_size = len(data)-training_size
train_data, test_data = data[0:training_size,:], data[training_size:len(data),:]

In [6]:
### Build input features with time lag of 1 day

def create_dataset(dataset, time_step = 1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i+time_step,0])
    return np.array(dataX), np.array(dataY)

time_step = 100
x_train, y_train = create_dataset(train_data, time_step)
x_test, y_test = create_dataset(test_data, time_step)

In [7]:
### Reshape the datasets into (#values, #time-steps, #1 dimensional output)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)

In [8]:
### Build different LSTM models

#!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

###### Prof's model
model = Sequential() 
model.add(LSTM(50, return_sequences = True, input_shape = (100,1))) 
model.add(LSTM(50, return_sequences = True)) 
model.add(LSTM(50)) 
model.add(Dense(1)) 
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.summary()

###### Modification: Change optimization function to Stochastic Gradient Descent (SGD)
model_mod = Sequential() 
model_mod.add(LSTM(50, return_sequences = True, input_shape = (100,1))) 
model_mod.add(LSTM(50, return_sequences = True)) 
model_mod.add(LSTM(50)) 
model_mod.add(Dense(1)) 
model_mod.compile(loss = 'mean_squared_error', optimizer = 'sgd')
model_mod.summary()

2022-03-18 22:05:34.958837: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (

In [ ]:
### Train LSTM models

lstm_history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 100, batch_size = 64, verbose = 1)
lstm_history_mod = model_mod.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 100, batch_size = 64, verbose = 1)

Epoch 1/100
13/13 [==============================] - 23s 831ms/step - loss: 0.0204 - val_loss: 0.0269
Epoch 2/100
13/13 [==============================] - 4s 324ms/step - loss: 0.0050 - val_loss: 0.0051
Epoch 3/100
13/13 [==============================] - 4s 319ms/step - loss: 0.0012 - val_loss: 0.0139
Epoch 4/100
13/13 [==============================] - 4s 340ms/step - loss: 6.4085e-04 - val_loss: 0.0130
Epoch 5/100
13/13 [==============================] - 5s 346ms/step - loss: 4.7139e-04 - val_loss: 0.0074
Epoch 6/100
13/13 [==============================] - 4s 315ms/step - loss: 4.3919e-04 - val_loss: 0.0067
Epoch 7/100
13/13 [==============================] - 4s 337ms/step - loss: 3.7109e-04 - val_loss: 0.0063
Epoch 8/100
13/13 [==============================] - 4s 304ms/step - loss: 3.6419e-04 - val_loss: 0.0055
Epoch 9/100
13/13 [==============================] - 4s 326ms/step - loss: 3.5821e-04 - val_loss: 0.0055
Epoch 10/100
13/13 [==============================] - 4s 336ms/ste

13/13 [==============================] - 4s 337ms/step - loss: 1.8516e-04 - val_loss: 0.0024
Epoch 79/100
13/13 [==============================] - 6s 520ms/step - loss: 2.1171e-04 - val_loss: 0.0020
Epoch 80/100
13/13 [==============================] - 5s 431ms/step - loss: 1.9288e-04 - val_loss: 0.0023
Epoch 81/100
13/13 [==============================] - 5s 391ms/step - loss: 1.8578e-04 - val_loss: 0.0027
Epoch 82/100
13/13 [==============================] - 6s 481ms/step - loss: 2.4825e-04 - val_loss: 0.0026
Epoch 83/100
13/13 [==============================] - 6s 467ms/step - loss: 1.8520e-04 - val_loss: 0.0020
Epoch 84/100
13/13 [==============================] - 6s 471ms/step - loss: 1.7776e-04 - val_loss: 0.0024
Epoch 85/100
13/13 [==============================] - 6s 459ms/step - loss: 1.7612e-04 - val_loss: 0.0020
Epoch 86/100
13/13 [==============================] - 4s 297ms/step - loss: 1.7161e-04 - val_loss: 0.0019
Epoch 87/100
13/13 [==============================] - 4s 31

In [ ]:
### Make predictions and inverse-map (0,1) to the true stock prices

def predictLSTM(model, train_input, test_input):
    train_predict = model.predict(train_input)
    test_predict = model.predict(test_input)
    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)
    return train_predict, test_predict

train_predict, test_predict = predictLSTM(model, x_train, x_test)
train_predict_mod, test_predict_mod = predictLSTM(model_mod, x_train, x_test)

In [ ]:
### Evaluate the LSTM model

print("Original model: Loss of training dataset", model.evaluate(x_train, y_train, verbose=0), 
      " Loss of test dataset", model.evaluate(x_test, y_test, verbose=0))
print("Modified model: Loss of training dataset", model_mod.evaluate(x_train, y_train, verbose=0), 
      " Loss of test dataset", model_mod.evaluate(x_test, y_test, verbose=0))

In [ ]:
### Plot predictions for test datasets

look_back = 100

def predict_plot(look_back, train_predict, test_predict):
    trainPredictPlot = np.empty_like(data)
    trainPredictPlot[:,:] = np.nan
    trainPredictPlot[look_back:len(train_predict)+look_back,:] = train_predict
    
    testPredictPlot=np.empty_like(data)
    testPredictPlot[:,:] = np.nan
    testPredictPlot[len(train_predict)+(look_back*2)+1:len(data)-1,:] = test_predict
    return trainPredictPlot, testPredictPlot

trainPredictPlot, testPredictPlot = predict_plot(look_back, train_predict, test_predict)
trainPredictPlot_mod, testPredictPlot_mod = predict_plot(look_back, train_predict_mod, test_predict_mod)

plt.figure(figsize = (12,8))
plt.plot(scaler.inverse_transform(data)[len(train_predict)+(look_back*2)+1:len(data)-1])
plt.plot(test_predict)
plt.title("Original model: Prediction")
plt.ylabel('HSI Index')
plt.show

plt.figure(figsize = (12,8))
plt.plot(scaler.inverse_transform(data)[len(train_predict_mod)+(look_back*2)+1:len(data)-1])
plt.plot(test_predict_mod)
plt.title("Modified model: Prediction")
plt.ylabel('HSI Index')
plt.show

In [ ]:
### Plot history for loss
loss_train = lstm_history.history['loss']
loss_train_mod = lstm_history_mod.history['loss']

epochs = range(1,101)

def chart_loss(loss_value, model_title):
    plt.plot(epochs, loss_value, 'b', label='Training loss')
    plt.title(model_title + ': Training loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

chart_loss(loss_train, "Original model")
chart_loss(loss_train_mod, "Modified model")